### Инициализация базы данных

In [94]:
import sqlite3 as sql
conn = sql.connect('db.db')
cur = conn.cursor()

### Функции

In [95]:
def get_adjacency_list(only_one_component : bool) -> dict:
    cur.execute(''' SELECT id FROM nodes ''')
    if only_one_component:
        ignore_list = [146669, 2562770, 2629691, 2635167, 2963597] #Великобритания, Ирландия и разные острова-государства
        v = []
        for el in cur.fetchall():
            if el[0] not in ignore_list:
                v.append(el)

        res = []
        for el in v:
            if el[0] not in ignore_list:
                cur.execute(''' SELECT target FROM edges WHERE source = ? ''',  (el[0], ))
                res.append([e[0] for e in cur.fetchall()])
    else:
        v = [el[0] for el in cur.fetchall()]

        res = []
        for el in v:
            cur.execute(''' SELECT target FROM edges WHERE source = ? ''',  (el, ))
            res.append([e[0] for e in cur.fetchall()])
    
    return {v[i] : res[i] for i in range(len(v))}

In [96]:
def get_center_and_k() -> {list, int}:
    ls : dict = get_adjacency_list(True)
    v = {el : 0 for el in ls.keys()}
    k = 0
    while len(ls) > 2:
        k+=1
        i = 0
        while i < len(ls.keys()):
            el = list(ls.keys())[i]
            if len(ls[el]) == 1:
                v[el] = k
                ls.pop(el)
                for a in ls.keys():
                    mas = set(a)
                    mas.discard(el)
                    ls[a] = list(mas)
            i+=1
    return (ls.keys(), k)

### 1) Изображение графа
![](Untitled.svg)

### 2) Данные

In [97]:
cur.execute(''' SELECT count(*) FROM nodes ''')
v = cur.fetchone()[0]
print(f"|V| = {v}")

cur.execute(''' SELECT count(*) FROM edges ''')
e = cur.fetchone()[0]//2
print(f"|E| = {e}")

cur.execute(''' SELECT
	label,
	(SELECT count(*) FROM edges WHERE edges.source = nodes.id) FROM nodes ''')
degrees = cur.fetchall()
min_deg = 1000
max_deg = -1
for el in degrees:
    min_deg = min(min_deg, el[1])
    max_deg = max(max_deg, el[1])
print(f"min deg = {min_deg}")
print(f"max deg = {max_deg}")

centers, k = get_center_and_k()
print("Центры:", end=' ')
for c in centers:
    cur.execute(''' SELECT label FROM nodes WHERE id=? ''', c)
    print(cur.fetchone()[0], end=' ')
print()

diametr = 2*k
if len(centers) == 1:
    diametr-=2
else:
    diametr-=1
print(f"Диаметр: {diametr}")

radius = abs((diametr+1)//2)
print(f"Радиус: {radius}")

components = 5 # {Великобритания + Ирландия}, разные острова-государства, {остальные страны}
cycle_num = e - v + components
print(f"Цикломатическое число = {cycle_num}")

|V| = 47
|E| = 86
min deg = 0
max deg = 9
Центры: Russia 
Диаметр: 8
Радиус: 4
Цикломатическое число = 44


### 3) Раскраска графа и хроматическое число
![](image.png)

Хроматическое число - 4

### 4) Компоненты связанности
![](image2.png)

### 5) Висячие вершины
Великобритания, Ирландия, Португалия, Гибралтар, Ватикан, Сан Марино, Дания, Монако

### 6) Двудольность
Граф недвудольный, например Испания, Франция и Андорра

### 7) Компоненты реберной двусвязности

![](image5.png)

### 8) Компоненты вершинной двусвязности, граф блоков и точек сочленения

![](image8.png)
![](image8a.png)

### 9) Эйлеров путь

Не существует, так как в графе присутствуют висячие вершины

### 10) Гамильтонов путь

Не существует, так как в графе присутствуют висячие вершины

### 11) Минимальное остовное дерево

![](image11.png)
![](image11a.png)

### 12) Максимальная клика

- France - Belgium - Luxembourg - Germany
- Serbia - Croatia - Montenegro - Bosnia and Herzegovina
- Russia - Latvia - Lithuania - Belarus

### 13) Код Прюфера

Germany, Latvia, Spain, North Macedonia, Montenegro, Francec, Sweden, Spain, Italy, Andorra, France, Belarus, Finland, Russia, Belarus, Ukraine, Poland, Germany, Slovakia, Germany, Netherlands, Belgium, Luxembourg, France, Switzerland, Austria, Liechtencstein, Switzerland, Moldova, Romania, Bulgaria, North Macedonia, Albania, Montenegro, Bosnia and Herzegovina